In [0]:
# Databricks notebook source
# 99_simulation_controller
# Descrição: Orquestra a simulação de 3 meses de dados (Nov/Dez/Jan)

import time

In [0]:
# 1. LIMPEZA TOTAL (Reset do Ambiente)
print("🧹 Limpando ambiente para simulação do zero...")
spark.sql("DROP TABLE IF EXISTS main.default.silver_customers")
spark.sql("DROP TABLE IF EXISTS main.default.gold_customers_by_city")
dbutils.fs.rm("/Volumes/main/default/lakehouse/bronze_customers", recurse=True) # Checkpoints e dados
dbutils.fs.rm("/Volumes/main/default/lakehouse/silver_customers", recurse=True)
dbutils.fs.rm("/Volumes/main/default/lakehouse/gold_customers_by_city", recurse=True)
# Opcional: Limpar a Raw também se quiser começar limpo, mas cuidado para não apagar pasta do volume
# dbutils.fs.rm("/Volumes/main/default/lakehouse/raw_customers", recurse=True) 

print("✨ Ambiente limpo!")
time.sleep(2)

In [0]:
# 2. DEFINIR A LINHA DO TEMPO
timeline = [
    "2025-11-01",
    "2025-12-01",
    "2026-01-01"
]

In [0]:
# 3. LOOP DE EXECUÇÃO
for date_sim in timeline:
    print(f"\n==========================================")
    print(f"🚀 INICIANDO SIMULAÇÃO: {date_sim}")
    print(f"==========================================")
    
    # A. Gerar Dados (Passando a data como parâmetro)
    print(f"1. Gerando dados para {date_sim}...")
    dbutils.notebook.run("00_generate_mock_data", 60, {"simulation_date": date_sim})
    
    # B. Ingestão Bronze
    print("2. Rodando Ingestão Bronze...")
    dbutils.notebook.run("01_ingest_bronze_autoloader", 120)
    
    # C. Processamento Silver (SCD2)
    print("3. Processando Silver (Histórico)...")
    dbutils.notebook.run("02_process_silver_scd2", 120)
    
    # D. Gold
    print("4. Atualizando KPIs Gold...")
    dbutils.notebook.run("03_gold_analytics", 60)
    
    print(f"✅ Mês {date_sim} concluído!")
    time.sleep(2) # Respiro

print("\n🎉 SIMULAÇÃO COMPLETA! Verifique o Dashboard agora.")